## Recency, Frequency, Monetary Value analysis

Understand customers based on their unique behavioral attributes.

### Group customers into cohorts and analyze their behavior over time.
In this chapter, we will dive into a very popular technique called RFM segmentation, which stands for Recency, Frequency and Monetary value segmentation.

### What is RFM segmentation?
Behavioral customer segmentation based on three metrics:
	- Recency (R): measure how recent was each customer's last purchase, 
	- Frequency (F): measure how many purchases the customer has done in the last 12 months
	- Monetary Value (M): measures how much has the customer spent in the last 12 months

we will use these values to assign customers to RFM segments. Once we have calculated these numbers, the next step is to group them into some sort of categorization such as high, medium and low. (Hay muchas formas de hacer esto)

### Grouping RFM values

The RFM values can be grouped in several ways:
	- Percentiles e.g. quantiles: We can break customers into groups of equal size based on percentile values of each metric
	- Pareto 80/20 cut: We can assign either high or low value to each metric based on a 80/20% Pareto split
	- Custom - based on business knowledge: we can use existing knowledge from previous business insights about certain threshold values for each metric

We are going to implement percentile-based grouping.

### Short review of percentiles

Process of calculating percentiles:
	1. Sort customers based on that metric
	2. Break customers into a pre-defined number of groups of equal size
	3. Assign a label to each group

Pandas function for calculating percentiles: qcut()

### Calculate percentiles with Python
Data with eight CustomerID and a randomly calculated Spend values.
We are going to implement percentile-based grouping.

In [1]:
data_example = {'CustomerID': [0,1,2,3,4,5,6,7], 'Spend': [137,335,172,355,303,233,244,229]}

In [2]:
import pandas as pd

data = pd.DataFrame(data_example)

In [3]:
data

,CustomerID,Spend
0,0,137
1,1,335
2,2,172
3,3,355
4,4,303
5,5,233
6,6,244
7,7,229


In [4]:
# Create a spend quartile with 4 groups and labels ranging from 1 through 4 
spend_quartile = pd.qcut(data['Spend'], q=4, labels=range(1,5))

# Assign the quartile values to the Spend_Quartile column in data
data['Spend_Quartile'] = spend_quartile

# Print data with sorted Spend values
print(data.sort_values('Spend'))

   CustomerID  Spend Spend_Quartile
0           0    137              1
2           2    172              1
7           7    229              2
5           5    233              2
6           6    244              3
4           4    303              3
1           1    335              4
3           3    355              4


**Calculate Recency deciles (q=10)**

We have created a dataset for you with random CustomerID and Recency_Days values as data. You will now use this dataset to group customers into quartiles based on Recency_Days values and assign labels to each of them.

Be cautious about the labels for this exercise. You will see that the labels are inverse, and will required one additional step in separately creating them. If you need to refresh your memory on the process of creating the labels, check out the slides!

In [6]:
data = pd.DataFrame({'CustomerID': [0,1,2,3,4,5,6,7], 'Recency_Days': [37,235,396,72,255,393,203,133]})

In [7]:
# Store labels from 4 to 1 in a decreasing order
r_labels = list(range(4, 0, -1))

# Create a spend quartile with 4 groups and pass the previously created labels 
recency_quartiles = pd.qcut(data['Recency_Days'], q=4, labels=r_labels)

# Assign the quartile values to the Recency_Quartile column in `data`
data['Recency_Quartile'] = recency_quartiles 

# Print `data` with sorted Recency_Days values
print(data.sort_values('Recency_Days'))

   CustomerID  Recency_Days Recency_Quartile
0           0            37                4
3           3            72                4
7           7           133                3
6           6           203                3
1           1           235                2
4           4           255                2
5           5           393                1
2           2           396                1


### Recency, Frequency, Monetary Value calculation

(Calculate metrics for each customer).

**Definitions**
	- Recency - it's the number of days since last customer transaction - the lower it is, the better, since every company wants ots customers to be recent and active. 
	- Frequency - calculates the number of transactions in the last 12 months, although there are variations such as average monthly transactions which depict the essence of this metric as well
	- Monetary Value - total spend in the last 12 months

One comment though - the 12 months is a standard way to do this, but it can be chosen arbitrarily depending on the business model and the lifecycle of the products and customers.


**Dataset and preparations**
	- Same online dataset like in the previous lessons
	- Need to do some data preparation before calculating the RFM value

In [8]:
import numpy as np
import pandas as pd
import datetime as dt

online = pd.read_csv('Online Retail.csv', sep=';')

# convert object to datetime
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'])

# convert object to float
online['UnitPrice'] = online['UnitPrice'].apply(lambda x: x.replace(',', '.'))
online['UnitPrice'] = online['UnitPrice'].apply(lambda col:pd.to_numeric(col, errors='coerce'))

In [9]:
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom


In [18]:
online = online.loc[(online['InvoiceDate'] >= '2010-12-10') & (online['InvoiceDate'] < '2011-12-13')]

In [19]:
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
25281,538365,22469,HEART OF WICKER SMALL,8,2010-12-12 10:11:00,1.65,17243.0,United Kingdom
25282,538365,84030E,ENGLISH ROSE HOT WATER BOTTLE,1,2010-12-12 10:11:00,4.25,17243.0,United Kingdom
25283,538365,22112,CHOCOLATE HOT WATER BOTTLE,3,2010-12-12 10:11:00,4.95,17243.0,United Kingdom
25284,538365,22835,HOT WATER BOTTLE I AM SO POORLY,5,2010-12-12 10:11:00,4.65,17243.0,United Kingdom
25285,538365,84029E,RED WOOLLY HOTTIE WHITE HEART.,4,2010-12-12 10:11:00,3.75,17243.0,United Kingdom


In [20]:
print('Min:{}; Max:{}'.format(min(online.InvoiceDate),
                              max(online.InvoiceDate)))

Min:2010-12-12 10:11:00; Max:2011-12-09 18:26:00


In [24]:
# Let's create a hypothetical snapshot_day data as if we're doing analysis recently
snapshot_date = max(online.InvoiceDate) + dt.timedelta(days=1)

In [25]:
snapshot_date

Timestamp('2011-12-10 18:26:00')

The online dataset has already been pre -processed and only includes the recent 12 months of data. In the real world, we would be working with the most recent snapshot of the data of today or yesterday, but in this case the data comes from 2010 and 2011, so we have to create hypothetical snapshot date that we'll use as a starting point to calculate metrics as if we're doing the analysis on the most recent data.

With days equal=1 argument we create a period of 1 day which we can then add to our date.

We aggregate the data on a Customer level, and calculate three metrics: we used the InvoceDate and pass it to the lambda function, and then take a difference between our snapshot date - which would be today in the real world - and the most recent or max() invoice date, this fives us the number of days between hypothetical today and the last transaction.

In [49]:
# Calculate Recency, Frequency and Monetary value for each customer 
datamart = online.groupby(['CustomerID']).agg([
    ('Recency', lambda x: (snapshot_date - x.max()).days),
    ('InvoiceNo', 'count'),
    ('TotalSum', 'sum')])

# Rename the columns 
datamart.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'TotalSum': 'MonetaryValue'}, inplace=True)

# Print top 5 rows
print(datamart.head())

ValueError: no results